# Brief Bot

## Required Dependencies


In [2]:
!pip install --upgrade ibm-watsonx-ai
!pip install -q git+https://github.com/openai/whisper.git
!pip install ffmpeg-python

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## files upload

In [3]:
from google.colab import files

print("📁 Upload audio/video file (MP3, WAV, MP4, etc.)")
uploaded = files.upload()
file_path = next(iter(uploaded))


📁 Upload audio/video file (MP3, WAV, MP4, etc.)


Saving Screen Recording 2025-06-28 115332.mp4 to Screen Recording 2025-06-28 115332.mp4


## Converting to WAV/MP3 (if video)

In [4]:
import os
import mimetypes
import ffmpeg

# auto-detect mimetype
mimetype = mimetypes.guess_type(file_path)[0]
base_name = os.path.splitext(file_path)[0]
audio_path = f"{base_name}.mp3"

if mimetype and mimetype.startswith("video"):
    print("🎥 Converting video to audio...")
    ffmpeg.input(file_path).output(audio_path, format='mp3', acodec='libmp3lame').run(overwrite_output=True)
else:
    audio_path = file_path  # already audio


🎥 Converting video to audio...


## Transcribe with Whisper

In [5]:
import whisper

model = whisper.load_model("base")
result = model.transcribe(audio_path)
transcript = result["text"]

print("✅ Transcript:")
print(transcript)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 88.2MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Transcript:
 Good morning, team. Let's quickly review the deliverables from this sprint. First, John, please finalize the budget proposal and send it to the finance department by Friday. Priya, you'll handle the onboarding of the two new hires. Make sure their accounts and training plans are ready before Monday. Also, the client feedback on the dashboard UI has been mostly positive, but they did request faster load times. Garage, can you work with the back end team to optimize the APIs this week? Let's aim to complete the demo recording for the investor pitch by Thursday. All right, that wraps up the key points. Any questions?


## Call IBM Granite for Summary

In [8]:
#  Use your actual API key and project ID here
API_KEY = "*****************************"
PROJECT_ID = "*******************************"
REGION = "us-south"

In [7]:
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai import Credentials
import re

credentials = Credentials(
    api_key=API_KEY,
    url=f"https://{REGION}.ml.cloud.ibm.com"
)

model = Model(
    model_id="ibm/granite-3-3-8b-instruct",
    credentials=credentials,
    project_id=PROJECT_ID,
    params={
        "decoding_method": "greedy",
        "max_new_tokens": 300,
        "stop_sequences": ["</response>"]
    }
)

prompt = f"""
<think>
You are a smart assistant. Given this meeting transcript, summarize the key points and action items with deadlines and owners.

Transcript:
{transcript}
</think>
<response>
"""

summary = model.generate_text(prompt)
cleaned = re.sub(r"[*#]+", "", summary)
cleaned = re.sub(r"\n{2,}", "\n", cleaned)
print("📋 Summary:\n", cleaned.strip())



/usr/local/lib/python3.11/dist-packages/ibm_watsonx_ai/foundation_models/model.py:105: DeprecationWarning: The `Model` class is deprecated and will be removed in a future release. Please use the `ModelInference` class instead. To update your imports, use: `from ibm_watsonx_ai.foundation_models import ModelInference`.
  warn(model_class_deprecated_warning, category=DeprecationWarning)


📋 Summary:
 Key Points and Action Items:
1. Budget Proposal:
   - Owner: John
   - Deadline: Friday
   - Action: Finalize and send the budget proposal to the finance department.
2. New Hire Onboarding:
   - Owner: Priya
   - Deadline: Before Monday
   - Action: Ensure new hires' accounts and training plans are ready.
3. Dashboard UI Feedback:
   - Status: Client feedback is mostly positive, but they requested faster load times.
   - Action: Address the request for improved load times.
4. API Optimization:
   - Owner: Garage (in collaboration with the back-end team)
   - Deadline: This week
   - Action: Work on optimizing the APIs to enhance load times.
5. Investor Pitch Demo Recording:
   - Deadline: Thursday
   - Action: Complete the demo recording for the investor pitch.
 Summary:
The meeting focused on finalizing deliverables from the current sprint. John is tasked with finalizing the budget proposal by Friday. Priya is responsible for preparing the onboarding of two new hires, ensu